In [25]:
from mingus.midi import fluidsynth as fs
import mingus.midi.midi_file_out as mfo
from mingus.containers.note_container import NoteContainer

In [26]:
fs.init("SoundFont/Giga Piano.sf2",'Direct Sound')

False

In [27]:
nc = NoteContainer(["A", "C", "E"])
mfo.write_NoteContainer("test.mid", nc)

True